# ECT 2026 Thailand Election — Google Drive File Extractor

Recursively list all files in the ECT 2026 election results Google Drive folder and export metadata as JSONL.

**Target folder**: `https://drive.google.com/drive/folders/1elYwd_ATWpm8q_ZxoxKcXO1TaVF7MElj`

**Output fields per file**:
| Field | Description |
|---|---|
| `province_name` | Thai province inferred from folder path |
| `path` | Full path from root folder |
| `size_mb` | File size in MB |
| `file_id` | Google Drive file ID |
| `folder_id` | Parent folder ID |

---

### Prerequisites

Run the following in a terminal **before** opening this notebook:

```bash
gcloud auth application-default login \
  --scopes="openid,https://www.googleapis.com/auth/userinfo.email,https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/drive.readonly"
```

Then enable the Drive API in your project:
```bash
gcloud services enable drive.googleapis.com
```

## 1. Install Dependencies

In [ ]:
!pip install -q google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 pandas

## 2. Authenticate & Build Drive Client

In [ ]:
import os
import json
import time
from pathlib import Path
from typing import Optional

import google.auth
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# ── Target folder ──────────────────────────────────────────────────────────────
ROOT_FOLDER_ID = "1elYwd_ATWpm8q_ZxoxKcXO1TaVF7MElj"

# ── Output ─────────────────────────────────────────────────────────────────────
OUTPUT_FILE = "datasets/ect_2026_drive_files.jsonl"

print(f"Root Folder ID : {ROOT_FOLDER_ID}")
print(f"Output File    : {OUTPUT_FILE}")

In [ ]:
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

# ── Option A: Application Default Credentials (ADC) ───────────────────────────
# Requires: gcloud auth application-default login --scopes="...drive.readonly"
try:
    credentials, project = google.auth.default(scopes=SCOPES)
    drive_service = build("drive", "v3", credentials=credentials)
    # Quick connectivity check
    drive_service.about().get(fields="user").execute()
    print(f"✅ Authenticated via ADC (project: {project})")
except Exception as e:
    print(f"⚠️  ADC failed: {e}")
    print()
    print("Run this in your terminal, then restart the kernel:")
    print()
    print('  gcloud auth application-default login \\')
    print('    --scopes="openid,https://www.googleapis.com/auth/userinfo.email,')
    print('              https://www.googleapis.com/auth/cloud-platform,')
    print('              https://www.googleapis.com/auth/drive.readonly"')
    raise

## 3. Recursive Traversal

In [ ]:
FOLDER_MIME = "application/vnd.google-apps.folder"


def list_folder_contents(service, folder_id: str) -> list[dict]:
    """Paginated listing of a Drive folder's direct children."""
    items, page_token = [], None
    while True:
        resp = (
            service.files()
            .list(
                q=f"'{folder_id}' in parents and trashed = false",
                fields="nextPageToken, files(id, name, mimeType, size, parents)",
                pageToken=page_token,
                pageSize=1000,
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
            )
            .execute()
        )
        items.extend(resp.get("files", []))
        page_token = resp.get("nextPageToken")
        if not page_token:
            break
    return items


def infer_province(path: str) -> Optional[str]:
    """
    Infer province from the folder path.
    ECT structure: <Root> / <Province> / ... / file
    Province is the second path component (index 1).
    """
    parts = [p.strip() for p in path.split("/") if p.strip()]
    return parts[1] if len(parts) >= 2 else None


def traverse(
    service,
    folder_id: str,
    current_path: str = "",
    results: Optional[list] = None,
    depth: int = 0,
) -> list[dict]:
    """Recursively collect file metadata from a Drive folder tree."""
    if results is None:
        results = []

    indent = "  " * depth
    for item in list_folder_contents(service, folder_id):
        name = item["name"]
        item_id = item["id"]
        path = f"{current_path}/{name}" if current_path else name

        if item["mimeType"] == FOLDER_MIME:
            print(f"{indent}📁 {name}/")
            traverse(service, item_id, path, results, depth + 1)
        else:
            size_bytes = int(item.get("size", 0))
            size_mb = round(size_bytes / (1024 * 1024), 4) if size_bytes else None
            results.append(
                {
                    "province_name": infer_province(path),
                    "path": path,
                    "size_mb": size_mb,
                    "file_id": item_id,
                    "folder_id": folder_id,
                }
            )
            print(f"{indent}📄 {name}  ({size_mb} MB)")

    return results


print("✅ Traversal functions ready")

## 4. Run Extraction

In [ ]:
# Resolve root folder name
root_meta = (
    drive_service.files()
    .get(fileId=ROOT_FOLDER_ID, fields="id,name", supportsAllDrives=True)
    .execute()
)
root_name = root_meta["name"]
print(f"Root: {root_name}  (id: {ROOT_FOLDER_ID})")
print("\n🔍 Traversing...\n")

t0 = time.time()
all_files = traverse(drive_service, ROOT_FOLDER_ID, current_path=root_name)
elapsed = time.time() - t0

print(f"\n{'─'*60}")
print(f"✅  {len(all_files)} files found in {elapsed:.1f}s")

## 5. Preview

In [ ]:
import pandas as pd

df = pd.DataFrame(all_files)
print(f"Total files    : {len(df)}")
print(f"Unique provinces: {df['province_name'].nunique()}")
print()
df.head(10)

In [ ]:
# Province-level summary
(
    df.groupby("province_name", dropna=False)
    .agg(file_count=("file_id", "count"), total_size_mb=("size_mb", "sum"))
    .assign(total_size_mb=lambda x: x["total_size_mb"].round(2))
    .sort_values("file_count", ascending=False)
    .reset_index()
)

## 6. Export JSONL

In [ ]:
out = Path(OUTPUT_FILE)
out.parent.mkdir(parents=True, exist_ok=True)

with out.open("w", encoding="utf-8") as f:
    for record in all_files:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"✅ {len(all_files)} records → {out}  ({out.stat().st_size / 1024:.1f} KB)")
print()
print("First 3 records:")
with out.open(encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        print(line.rstrip())